In [1]:
import pandas as pd
data=pd.read_excel("D:/META LAB/Excel/symp.xlsx")

In [2]:
!pip install deep-translator
from deep_translator import GoogleTranslator
!pip install autocorrect
from string import digits
import regex as re
from autocorrect import Speller
import nltk
import jieba
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import wordnet
import jieba
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zalramli\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zalramli\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zalramli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zalramli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
class M_PreProcess:

  def __init__(self,data):

    # Translate
    translations=[]
    # Deteksi indo, translate ke inggris
    translations = {}
    for column in data.columns:
        # Unique elements of the column
        unique_elements = data[column].unique()
    for element in unique_elements:
        # Adding all the translations to a dictionary (translations)
        translations[element] = GoogleTranslator(source='auto', target='en').translate(element)
    data.replace(translations, inplace = True)
    data=pd.DataFrame(data["q_11"])["q_11"]
    
    # Case Folding
    casefolded=[]
    for line in data:
      a=line.lower()
      casefolded.append(a)
    
    # Omit Number and Punctuations
    number=[]
    for line in casefolded:
      remove_digits=str.maketrans(' ', ' ', digits)
      result=line.translate(remove_digits)
      number.append(result)
    punct=[]
    for char in number:
      result=re.sub(r"\p{P}(?<!,)","", char)
      punct.append(result)
    
    # Split by Comma
    splitted=[]
    for line in punct:
      a=line.split(',')
      splitted.append(a)
    
    # Lemmatize 1
    def pos_tagger(nltk_tag):
      if nltk_tag.startswith('J'):
        return wordnet.VERB
      elif nltk_tag.startswith('V'):
        return wordnet.VERB
      elif nltk_tag.startswith('N'):
        return wordnet.NOUN
      elif nltk_tag.startswith('R'):
        return wordnet.ADV
      else:          
        return None
    
    lemmatizer = WordNetLemmatizer()
    pos_tagged=[]
    for i in range(len(splitted)):
      a=[]
      for j in range(len(splitted[i])):
        b=nltk.pos_tag(nltk.word_tokenize(splitted[i][j]))
        a.append(b)
      pos_tagged.append(a)
    
    wordnet_tagged=[]
    for i in range(len(pos_tagged)):
      a=[]
      for j in range(len(pos_tagged[i])):
        b=list(map(lambda x: (x[0], pos_tagger(x[1])),pos_tagged[i][j]))
        a.append(b)
      wordnet_tagged.append(a)
    
    lem = []
    for i in range(len(wordnet_tagged)):
      a=[]
      for j in range(len(wordnet_tagged[i])):
        b=[]
        for word, tag in wordnet_tagged[i][j]:
          if tag is None:
            b.append(word)
          else:
            b.append(lemmatizer.lemmatize(word, tag))
        b=" ".join(b)
        a.append(b)
      lem.append(a)

    # Lemmatize 2
    def pos_tagger(nltk_tag):
      if nltk_tag.startswith('J'):
        return wordnet.VERB
      elif nltk_tag.startswith('V'):
        return wordnet.VERB
      elif nltk_tag.startswith('N'):
        return wordnet.VERB
      elif nltk_tag.startswith('R'):
        return wordnet.ADV
      else:          
        return None
    
    pos_tagged=[]
    for i in range(len(lem)):
      a=[]
      for j in range(len(lem[i])):
        b=nltk.pos_tag(nltk.word_tokenize(lem[i][j]))
        a.append(b)
      pos_tagged.append(a)
   
    wordnet_tagged=[]
    for i in range(len(pos_tagged)):
      a=[]
      for j in range(len(pos_tagged[i])):
        b=list(map(lambda x: (x[0], pos_tagger(x[1])),pos_tagged[i][j]))
        a.append(b)
      wordnet_tagged.append(a)
    
    lem2 = []
    for i in range(len(wordnet_tagged)):
      a=[]
      for j in range(len(wordnet_tagged[i])):
        b=[]
        for word, tag in wordnet_tagged[i][j]:
          if tag is None:
            b.append(word)
          else:
            b.append(lemmatizer.lemmatize(word, tag))
        b=" ".join(b)
        a.append(b)
      lem2.append(a)

    # Remove Stopwords 1
    stopwords = nltk.corpus.stopwords.words('english')
    removestop = [] 
    for i in range(len(lem2)):
      a=[]
      for j in range(len(lem2[i])):
        word_token=nltk.word_tokenize(lem2[i][j])
        word_token=[word for word in word_token if not word in stopwords]
        a.append(" ".join(word_token))
      removestop.append(a)

    syn1 = pd.read_csv("D:/META LAB/Drive/list_temp_lemmed (1).csv",header=None,sep=" ",error_bad_lines=False)
    sin_direct2= syn1[syn1.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)

    # Remove Stopwords 2
    for n,line in enumerate(sin_direct2):
      if line.startswith("line"):
        sin_direct2[n] = " "+line.rstrip()
      else:
        sin_direct2[n]=line.rstrip()
    sw1=','.join(sin_direct2)
    sw1=sw1.split(',')

    removestop2 = []
    for i in range(len(removestop)):
      a=[]
      for j in range(len(removestop[i])):
        word_token=nltk.word_tokenize(removestop[i][j])
        word_token=[word for word in word_token if word in sw1]
        a.append(" ".join(word_token))
      removestop2.append(a)

# Synonym Replacemnet 1
    def tihuan_tongyici(string1):
      combine_dict = {}
      for line in open("D:/META LAB/Drive/list_temp_lemmed.txt", "r"):
        seperate_word = line.strip().split(",")
        num = len(seperate_word)
        for i in range(1, num):
            combine_dict[seperate_word[i]] = seperate_word[0]    
      
      f = string1
      
      final_sentence = ""
      for word in f.split(", "):
        if word in combine_dict:
          word = combine_dict[word]
          final_sentence += word
        else:
          final_sentence += word
      return final_sentence

    syno = []
    for i in range(len(removestop2)):
      a=[]
      for j in range(len(removestop2[i])):
        word_token=tihuan_tongyici(removestop2[i][j])
        a.append("".join(word_token))
      syno.append(a)
      
# Synonym Replacemnet 2
    def tihuan_tongyici1(string1):
      combine_dict = {}
      for line in open("D:/META LAB/Drive/list_temp_lemmed.txt", "r"):
        seperate_word = line.strip().split(",")
        num = len(seperate_word)
        for i in range(1, num):
            combine_dict[seperate_word[i]] = seperate_word[0]    
      
      seg_list = jieba.cut(string1, cut_all = False)
      f = "/".join(seg_list) # Without utf-8 encoding, it can't correspond to the words in the tongyici file.
      final_sentence = ""
      for word in f.split("/"):
        if word in combine_dict:
            word = combine_dict[word]
            final_sentence += word
        else:
            final_sentence += word
    # print final_sentence
      return final_sentence

    syno1 = []
    for i in range(len(syno)):
      a=[]
      for j in range(len(syno[i])):
        word_token=tihuan_tongyici1(syno[i][j])
        a.append("".join(word_token))
      syno1.append(a)

    syn = pd.read_csv("D:/META LAB/Drive/list_temp_lemmed.csv",header=None,sep="_",error_bad_lines=False)
    sin_direct= syn[syn.columns[0:]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1)
          
#remove stopwords 3
    for n,line in enumerate(sin_direct):
      if line.startswith("line"):
        sin_direct[n] = " "+line.rstrip()
      else:
        sin_direct[n]=line.rstrip()
    sw2=','.join(sin_direct)
    sw2=sw2.split(',')

    removestop3 = []
    for i in range(len(syno1)):
      a=[]
      for j in range(len(syno1[i])):
        word_token=nltk.word_tokenize(syno1[i][j])
        word_token=[word for word in word_token if word in sw2]
        a.append(" ".join(word_token))
      removestop3.append(a)
    
    # Join
    joined=[]
    for line in removestop3:
      a=" ".join(line)
      joined.append(a)
    
    # Vectorize
    tf_vectorizer = CountVectorizer(max_df=len(joined), 
                                     min_df=0,
                                     decode_error='ignore',binary=True)
    tf = tf_vectorizer.fit_transform(joined)
    self.symptoms = pd.DataFrame(tf.toarray(), columns=tf_vectorizer.get_feature_names())

In [4]:
x=M_PreProcess(data)
x.symptoms

C:\Users\zalramli\AppData\Local\Temp/ipykernel_11980/1234393015.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  x=M_PreProcess(data)
b'Skipping line 8: expected 46 fields, saw 56\n'
Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\zalramli\AppData\Local\Temp\jieba.cache
Loading model cost 1.449 seconds.
Prefix dict has been built successfully.
b'Skipping line 8: expected 25 fields, saw 29\n'


,addict,alcohol,anger,anxious,appetite,balance,breathe,communicate,concentrate,confuse,...,suicide,sweat,tire,trauma,tremble,unconscious,violence,weight,withdrawal,worry
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
153,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
154,0,0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0


In [11]:
x.symptoms.to_excel ("D:/META LAB/Excel/symp_fix.xlsx", index = False, header=True)